#Data Cleaning and Transformation

In [ ]:
#query to union historical data with daily data + group by ticker and date
CREATE TABLE `stock_data.stock_prices_combined` AS
SELECT *
FROM (
    Select
      ticker,
      TIMESTAMP(date) AS date,
      open_price,
      close_price,
      high_price,
      low_price,
      volume
    FROM `stock_data.daily_data`

    UNION ALL

   SELECT
      ticker,
      TIMESTAMP(date) AS date,
      open_price,
      close_price,
      high_price,
      low_price,
      volume
    FROM `stock_data.historical_data`
)
Order BY
  ticker,
  date;

In [ ]:
## Add Derived Columns (Price Change + Profit/Loss)
CREATE VIEW `stock_data.v_stock_prices_transformed` AS
SELECT
  *,
  close_price - open_price AS price_change,
  CASE
    WHEN close_price - open_price > 0 THEN 'Profit'
    WHEN close_price - open_price < 0 THEN 'Loss'
    ELSE 'No Change'
  END AS profit_or_loss
FROM `stock_data.stock_data_combined`;


In [ ]:
#Add missing data to ticker_details
UPDATE stock_data.ticker_details
SET
  market_cap = CASE
                 WHEN ticker = 'MSFT' THEN 2937000000000.0
                 WHEN ticker = 'JNJ' THEN 395170000000.0
               END,
  description = CASE
                  WHEN ticker = 'MSFT' THEN 'Microsoft is a global technology leader known for its Windows operating system, Microsoft Office suite, Azure cloud platform, and advancements in artificial intelligence. The company continues to expand its presence in cloud computing, AI, and enterprise services.'
                  WHEN ticker = 'JNJ' THEN 'Founded in 1886, Johnson & Johnson is a multinational corporation specializing in pharmaceuticals, biotechnology, and medical technologies. The company has a global workforce and focuses on developing and producing prescription drugs and medical device technologies.'
                END,
  homepage_url = CASE
                   WHEN ticker = 'MSFT' THEN 'https://www.microsoft.com'
                   WHEN ticker = 'JNJ' THEN 'https://www.jnj.com'
                 END,
  total_employees = CASE
                      WHEN ticker = 'MSFT' THEN 228000
                      WHEN ticker = 'JNJ' THEN 138100
                    END,
  list_date = CASE
                WHEN ticker = 'MSFT' THEN TIMESTAMP('1986-03-13 00:00:00')
                WHEN ticker = 'JNJ' THEN TIMESTAMP('1944-01-01 00:00:00')
              END
WHERE ticker IN ('MSFT', 'JNJ');

#add missing data to ticker_details_latest
UPDATE stock_data.ticker_details_latest
SET
  market_cap = CASE
                 WHEN ticker = 'ORCL' THEN 371140000000.0
                 WHEN ticker = 'JNJ' THEN 365650000000.0
               END,
  description = CASE
                  WHEN ticker = 'ORCL' THEN 'Oracle is a global leader in enterprise software, cloud infrastructure, and database technology. The company offers a comprehensive suite of applications and services, including Oracle Cloud Infrastructure (OCI), and has been expanding its capabilities in artificial intelligence and data security.'
                  WHEN ticker = 'JNJ' THEN 'Founded in 1886, Johnson & Johnson is a multinational corporation specializing in pharmaceuticals, biotechnology, and medical technologies. The company has a global workforce and focuses on developing and producing prescription drugs and medical device technologies.'
                END,
  homepage_url = CASE
                   WHEN ticker = 'ORCL' THEN 'https://www.oracle.com/'
                   WHEN ticker = 'JNJ' THEN 'https://www.jnj.com'
                 END,
  total_employees = CASE
                      WHEN ticker = 'ORCL' THEN 159000
                      WHEN ticker = 'JNJ' THEN 138100
                    END,
  list_date = CASE
                WHEN ticker = 'ORCL' THEN TIMESTAMP('1986-03-12 00:00:00')
                WHEN ticker = 'JNJ' THEN TIMESTAMP('1944-01-01 00:00:00')
              END
WHERE ticker IN ('ORCL', 'JNJ');


In [ ]:
#join ticker_details with ticker_details_latest + get difernece in market cap
SELECT
  feb.ticker,
  feb.name,
  feb.market_cap AS market_cap_feb,
  apr.market_cap AS market_cap_apr,
  (apr.market_cap - feb.market_cap) AS market_cap_diff,
  ROUND((apr.market_cap - feb.market_cap) / feb.market_cap * 100, 2) AS percent_change
FROM `project.dataset.ticker_details_feb` AS feb
JOIN `project.dataset.ticker_details_apr` AS apr
  ON feb.ticker = apr.ticker

#Business Questions

In [ ]:
#Which stocks have shown the highest average daily profit over the last month?
SELECT
  ticker,
  avg(price_change) as average_price_change
FROM `stock_data.stock_prices_transformed`
WHERE date >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 30 DAY)
GROUP BY ticker
ORDER BY average_price_change DESC
LIMIT 10;

In [ ]:
#What is the volatility (high - low) trend for each stock over time?
SELECT
  ticker,
  DATE(date) AS trade_date,
  AVG(high_price - low_price) AS avg_volatility
FROM `stock_data.stock_prices_transformed`
GROUP BY ticker, trade_date
ORDER BY trade_date;

In [ ]:
#Top 5 Stocks by Market Cap That Consistently Show Profits
WITH profit_stats AS (
  SELECT
    s.ticker,
    COUNTIF(profit_or_loss = 'Profit') AS profit_days,
    COUNT(*) AS total_days,
    t.market_cap
  FROM `stock_data.v_stock_prices_transformed` s
  JOIN `stock_data.ticker_details` t
    ON s.ticker = t.ticker
  GROUP BY s.ticker, t.market_cap
)

SELECT
  ticker,
  profit_days,
  total_days,
  ROUND(profit_days / total_days, 2) AS profit_ratio,
  market_cap
FROM profit_stats
ORDER BY profit_ratio DESC, market_cap DESC
LIMIT 5;

In [ ]:
#What are the most traded stocks in terms of volume over the month of March
SELECT
  ticker,
  SUM(volume) AS total_volume
FROM `stock_data.daily_data`
WHERE date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
GROUP BY ticker
ORDER BY total_volume DESC
LIMIT 10;


In [ ]:
#Which companies had >5% daily drop in stock price in March?
SELECT
  ticker,
  date,
  open_price,
  close_price,
  ROUND((close_price - open_price) / open_price * 100, 2) AS percent_change
FROM `stock_data.daily_data`
WHERE date >= DATE_SUB(CURRENT_DATE(), INTERVAL 13 DAY)
  AND (close_price - open_price) / open_price <= -0.05
ORDER BY percent_change ASC;


In [ ]:
#Detecting Bearish Signals (Price Down, Volume Up)
WITH ranked_prices AS (
  SELECT
    ticker,
    DATE(date) AS trade_date,
    close_price,
    volume,
    ROW_NUMBER() OVER (PARTITION BY ticker ORDER BY DATE(date)) AS rn
  FROM `stock_data.v_stock_prices_transformed`
),

joined_prices AS (
  SELECT
    curr.ticker,
    curr.trade_date,
    curr.close_price AS curr_close,
    prev.close_price AS prev_close,
    curr.volume AS curr_volume,
    prev.volume AS prev_volume
  FROM ranked_prices curr
  JOIN ranked_prices prev
    ON curr.ticker = prev.ticker
    AND curr.rn = prev.rn + 1
)

SELECT
  ticker,
  trade_date,
  prev_close,
  curr_close,
  prev_volume,
  curr_volume,
  ROUND((curr_close - prev_close), 2) AS price_change,
  ROUND((curr_volume - prev_volume), 2) AS volume_change
FROM joined_prices
WHERE curr_close < prev_close  -- price dropped
  AND curr_volume > prev_volume  -- volume increased
ORDER BY trade_date DESC, ticker;


In [ ]:
#Which days had unusually high trading volume for any stock
WITH daily_market_volume AS (
  SELECT
    DATE(date) AS trade_date,
    SUM(volume) AS total_volume
  FROM `stock_data.v_stock_prices_transformed`
  GROUP BY trade_date
),

volume_threshold AS (
  SELECT
    PERCENTILE_CONT(total_volume, 0.95) OVER () AS volume_spike_threshold
  FROM daily_market_volume
  LIMIT 1
)

SELECT
  d.trade_date,
  d.total_volume
FROM daily_market_volume d
JOIN volume_threshold v
  ON d.total_volume > v.volume_spike_threshold
ORDER BY d.total_volume DESC;
